# Importing packages

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import random
import pandas as pd

# GridWorld

In [67]:
class GridWorld:
    def __init__(self, width, height, discount_factor, actions_description, eta,
                 initial_state, goal_states, off_grid_reward, goal_reward, int_reward,
                neg_goal_states=None, neg_reward=None):
        '''
            Create GridWorld and calculate optimal Q values
            
            ---------
            
            Arguments:
            width : width of GridWorld
            height : height of GridWorld
            discount_factor : discount factor in the definition of V and Q values
            actions_description : 2-tuple, first parameter is a mapping from state and action to a new state, 
            second parameter is the number of possible actions available for the agent
            eta : learning rate in Q-learning algorithm
            initial_state : initial state from which the agent starts its movement
            goal_states : list of goal states
            off_grid_reward : reward for moving off GridWorld
            goal_reward : reward for achieving the goal state
            int_reward : reward for any other available actions  
            neg_goal_states : list of goal states with negative reward when reaching
            neg_reward : negative reward when reaching neg_goal_states
        '''
        self.width = width
        self.height = height
        self.discount_factor = discount_factor
        self.action, self.n_possible_actions = actions_description
        self.eta = eta
        self.Q = np.zeros((self.n_possible_actions, self.height, self.width))
        self.V = np.zeros((self.height, self.width))
        self.initial_state = initial_state
        self.goal_states = goal_states
        self.off_grid_reward = off_grid_reward
        self.goal_reward = goal_reward
        self.int_reward = int_reward
        self.temperature = 10.0
        self.neg_goal_states = neg_goal_states
        self.neg_reward = neg_reward
        
    def choose_action(self):
        state_y, state_x = self.current_state
        q = self.Q[:, state_y, state_x] / self.temperature
        probs = np.exp(q)
        probs /= probs.sum()
        experiment = np.random.multinomial(1, probs)
        self.temperature *= 0.99
        return np.where(experiment == 1)[0][0]      
    
    def off_grid_check(self, state):
        y, x = state
        
        if x < 0:
            return True
        
        if x >= self.width:
            return True
        
        if y < 0:
            return True
        
        if y >= self.height:
            return True
        
        return False
    
    def update_V(self):
        self.V = np.max(self.Q, axis=0)
        
    def zeroize_Q(self):
        self.Q = np.zeros((self.n_possible_actions, self.height, self.width))
        
    def optimal_policy(self):
        return np.argmax(self.Q, axis=0)
        
    def Q_learning(self):
        self.current_state = self.initial_state
        self.temperature = 10.0
        while self.current_state not in self.goal_states:
            action_num = self.choose_action()
            new_state = self.action(self.current_state, action_num)
            y, x = self.current_state
            if self.off_grid_check(new_state):
                self.Q[action_num, y, x] += self.eta * (self.off_grid_reward + 
                                                        self.discount_factor * np.max(self.Q[:, y, x]) - 
                                                        self.Q[action_num, y, x])
            else:
                reward = self.int_reward
                if (new_state in self.goal_states):
                    reward = self.goal_reward
                    
                y_n, x_n = new_state
                self.Q[action_num, y, x] += self.eta * (reward + 
                                                        self.discount_factor * np.max(self.Q[:, y_n, x_n]) - 
                                                        self.Q[action_num, y, x])
                self.current_state = new_state
        self.update_V()
        
    def Sarsa(self):
        self.current_state = self.initial_state
        self.temperature = 10.0
        action_num = self.choose_action()
        while self.current_state not in self.goal_states:
            new_state = self.action(self.current_state, action_num)
            y, x = self.current_state
            if self.off_grid_check(new_state):
                new_action_num = self.choose_action()
                self.Q[action_num, y, x] += self.eta * (self.off_grid_reward + 
                                                        self.discount_factor * self.Q[new_action_num, y, x] - 
                                                        self.Q[action_num, y, x])
            else:
                reward = self.int_reward
                if (new_state in self.goal_states):
                    reward = self.goal_reward
                    
                y_n, x_n = new_state
                self.current_state = new_state
                new_action_num = self.choose_action()
                self.Q[action_num, y, x] += self.eta * (reward + 
                                                        self.discount_factor * self.Q[new_action_num, y_n, x_n] - 
                                                        self.Q[action_num, y, x])
            action_num = new_action_num
        self.update_V()     
    
    def Q_learning_negstatesallowed(self):
        self.current_state = self.initial_state
        self.temperature = 10.0
        while (self.current_state not in self.goal_states and self.current_state not in self.neg_goal_states):
            action_num = self.choose_action()
            new_state = self.action(self.current_state, action_num)
            y, x = self.current_state
            if self.off_grid_check(new_state):
                self.Q[action_num, y, x] += self.eta * (self.off_grid_reward + 
                                                        self.discount_factor * np.max(self.Q[:, y, x]) - 
                                                        self.Q[action_num, y, x])
            else:
                reward = self.int_reward
                if (new_state in self.goal_states):
                    reward = self.goal_reward
                if (new_state in self.neg_goal_states):
                    reward = self.neg_reward
                    
                y_n, x_n = new_state
                self.Q[action_num, y, x] += self.eta * (reward + 
                                                        self.discount_factor * np.max(self.Q[:, y_n, x_n]) - 
                                                        self.Q[action_num, y, x])
                self.current_state = new_state
        self.update_V()



In [3]:
def action(state, number):
    y, x = state
    if number == 0: # up
        return (y - 1, x)
    if number == 1: # down
        return (y + 1, x)
    if number == 2: # left
        return (y, x - 1)
    if number == 3: # right
        return (y, x + 1)

# Q learning 

In [14]:
GW = GridWorld(5, 4, 0.9, (action, 4), 0.05, (0, 0), [(3, 4)], -1.0, 100.0, 0.0)

In [15]:
n_episodes = 100000
for i in range(n_episodes):
    GW.Q_learning()

In [16]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
GW.Q

array([[[46.830, 52.144, 58.049, 64.610, 1.836],
        [47.830, 53.144, 59.049, 3.506, 1.152],
        [53.144, 59.049, 65.610, 0.011, 0.208],
        [59.049, 65.610, 0.812, 7.897, 0.000]],

       [[53.144, 59.049, 65.610, 72.900, 33.497],
        [59.049, 65.610, 72.900, 81.000, 90.000],
        [65.610, 72.900, 81.000, 90.000, 100.000],
        [64.610, 71.900, 80.000, 0.341, 0.000]],

       [[46.830, 47.830, 53.144, 59.049, 3.693],
        [52.144, 53.144, 59.049, 3.469, 0.059],
        [58.049, 59.049, 65.610, 13.575, 4.050],
        [1.169, 65.610, 3.847, 7.897, 0.000]],

       [[53.144, 59.049, 65.610, 7.270, 1.794],
        [59.049, 65.610, 72.900, 81.000, 3.177],
        [65.610, 72.900, 81.000, 90.000, 16.961],
        [72.900, 81.000, 90.000, 100.000, 0.000]]])

In [18]:
GW.optimal_policy()

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [3, 3, 3, 3, 0]])

In [32]:
pd.DataFrame(GW.Q[0]).round(2)

,0,1,2,3,4
0,46.83,52.14,58.05,64.61,1.84
1,47.83,53.14,59.05,3.51,1.15
2,53.14,59.05,65.61,0.01,0.21
3,59.05,65.61,0.81,7.90,0.00


In [29]:
pd.DataFrame(GW.Q[1]).round(3)

,0,1,2,3,4
0,53.144,59.049,65.61,72.900,33.497
1,59.049,65.610,72.90,81.000,90.000
2,65.610,72.900,81.00,90.000,100.000
3,64.610,71.900,80.00,0.341,0.000


In [30]:
pd.DataFrame(GW.Q[2]).round(3)

,0,1,2,3,4
0,46.830,47.830,53.144,59.049,3.693
1,52.144,53.144,59.049,3.469,0.059
2,58.049,59.049,65.610,13.575,4.050
3,1.169,65.610,3.847,7.897,0.000


In [34]:
pd.DataFrame(GW.Q[3]).round(2)

,0,1,2,3,4
0,53.14,59.05,65.61,7.27,1.79
1,59.05,65.61,72.90,81.00,3.18
2,65.61,72.90,81.00,90.00,16.96
3,72.90,81.00,90.00,100.00,0.00


In [35]:
pd.DataFrame(GW.V).round(2)

,0,1,2,3,4
0,53.14,59.05,65.61,72.9,33.5
1,59.05,65.61,72.90,81.0,90.0
2,65.61,72.90,81.00,90.0,100.0
3,72.90,81.00,90.00,100.0,0.0


# Sarsa

In [36]:
GW = GridWorld(5, 4, 0.9, (action, 4), 0.05, (0, 0), [(3, 4)], -1.0, 100.0, 0.0)

In [43]:
n_episodes = 500000
for i in range(n_episodes):
    GW.Sarsa()

In [44]:
pd.DataFrame(GW.Q[0]).round(2)

,0,1,2,3,4
0,33.37,40.13,46.54,54.10,57.33
1,33.84,41.11,48.39,55.02,60.40
2,38.16,47.69,56.90,66.02,72.64
3,45.76,54.98,65.73,75.00,0.00


In [45]:
pd.DataFrame(GW.Q[1]).round(2)

,0,1,2,3,4
0,38.59,47.24,55.73,65.66,72.26
1,45.20,54.62,64.83,75.27,86.32
2,48.46,60.57,73.50,86.96,100.00
3,49.96,59.79,71.32,85.34,0.00


In [46]:
pd.DataFrame(GW.Q[2]).round(2)

,0,1,2,3,4
0,33.46,33.67,41.21,47.55,54.69
1,38.66,39.26,46.54,55.95,62.82
2,43.41,43.98,54.35,64.30,75.48
3,47.75,49.66,59.60,71.39,0.00


In [47]:
pd.DataFrame(GW.Q[3]).round(2)

,0,1,2,3,4
0,40.48,48.81,55.81,62.30,60.58
1,46.53,55.98,65.36,74.13,69.00
2,54.45,64.65,75.06,84.21,85.39
3,61.94,73.38,85.40,100.00,0.00


In [48]:
pd.DataFrame(GW.V).round(2)

,0,1,2,3,4
0,40.48,48.81,55.81,65.66,72.26
1,46.53,55.98,65.36,75.27,86.32
2,54.45,64.65,75.06,86.96,100.00
3,61.94,73.38,85.40,100.00,0.00


In [50]:
GW.optimal_policy()

array([[3, 3, 3, 1, 1],
       [3, 3, 3, 1, 1],
       [3, 3, 3, 1, 1],
       [3, 3, 3, 3, 0]])

# Two goal states

In [64]:
GW = GridWorld(5, 4, 0.9, (action, 4), 0.05, (0, 0), [(3, 4), (0, 4)], -1.0, 100.0, 0.0)

In [65]:
n_episodes = 100000
for i in range(n_episodes):
    GW.Q_learning()

In [66]:
GW.optimal_policy()

array([[3, 3, 3, 3, 0],
       [0, 0, 0, 3, 0],
       [0, 0, 0, 1, 1],
       [0, 3, 3, 3, 0]])

In [63]:
pd.DataFrame(GW.Q[3]).round(2)

,0,1,2,3,4
0,72.90,81.00,90.00,100.00,0.00
1,65.61,72.90,81.00,30.89,1.08
2,65.61,72.90,81.00,90.00,0.30
3,59.05,65.61,3.82,22.62,0.00


# GridWorld with negative states

In [68]:
GW_n = GridWorld(5, 4, 0.9, (action, 4), 0.05, (0, 0), [(3, 4), (0, 4)], -1.0, 100.0, 0.0, [(0, 3)], -100.0)

In [69]:
n_episodes = 100000
for i in range(n_episodes):
    GW_n.Q_learning_negstatesallowed()

In [70]:
GW_n.optimal_policy()

array([[1, 1, 1, 0, 0],
       [3, 3, 3, 3, 0],
       [1, 1, 1, 1, 1],
       [3, 3, 3, 3, 0]])